### Playing with `claude-v1.3-100k`

In [1]:
import os
import anthropic
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import openai
import time


In [2]:
client = anthropic.Client(api_key=os.environ['ANTHROPIC_API_KEY'])

In [6]:
class ContextualQA:
    def __init__(self, client, model="claude-v1.3-100k"):
        self.client = client
        self.model = model
        self.context = ""
        self.questions = []
        self.responses = []

    def load_text(self, text):
        self.context = text

    def ask_question(self, question):
        leading_prompt = "Consider the text document below:"
        trailing_prompt = "Now answer the following question:"
        prompt = f"{anthropic.HUMAN_PROMPT} {leading_prompt}\n\n{self.context}\n\n{trailing_prompt}\n\n{anthropic.HUMAN_PROMPT} {question} {anthropic.AI_PROMPT}"
        response = self.client.completion_stream(
            prompt=prompt,
            stop_sequences=[anthropic.HUMAN_PROMPT],
            max_tokens_to_sample=6000,
            model=self.model,
            stream=False,
        )
        responses = [data for data in response]
        self.questions.append(question)
        self.responses.append(responses)
        return responses

    def clear_context(self):
        self.context = ""
        self.questions = []
        self.responses = []

    def __getstate__(self):
        state = self.__dict__.copy()
        del state["client"]
        return state

    def __setstate__(self, state):
        self.__dict__.update(state)
        self.client = None


In [7]:
# Initialize the QA system
qa_system = ContextualQA(client)

# Load a text
with open('./gpts_are_gpts/main.tex', 'r') as f:
    paper_1 = f.read()


In [8]:
# Create a QA system
qa_system.load_text(paper_1)

# Ask a question
responses = qa_system.ask_question("What is the main argument of the paper?")
print(responses[0]['completion'])

responses = qa_system.ask_question("Can you summerize the paper in a few sentences?")
print(responses[0]['completion'])

responses = qa_system.ask_question("Can you tell me how GPT will change the world?")
print(responses[0]['completion'])

responses = qa_system.ask_question("Can you tell me the benefits of GPT on society and economy?")
print(responses[0]['completion'])


 The main argument of the paper is that large language models (LLMs), such as Generative Pre-trained Transformers (GPTs), exhibit traits of general-purpose technologies and could have considerable economic, social and policy implications. The paper provides evidence for this argument in several ways:

1) The authors develop a new rubric for assessing the exposure of occupations and tasks to LLM capabilities. Using this rubric, they find that most occupations in the U.S. labor market have at least some degree of exposure to LLMs, with higher wage occupations generally exhibiting higher exposure. 

2) The authors show that LLMs meet two key criteria for general-purpose technologies: they are improving over time and have the potential for widespread impact across the economy. The paper documents the rapid progress in LLM capabilities in recent years. And the authors' analysis of occupational exposure suggests LLMs could significantly impact a diverse range of jobs.

3) The authors argue t

In [13]:
print(qa_system.responses[-1][0]['completion'])

 Here are some of the potential benefits of GPTs and large language models on society and the economy:

•Increased productivity. By automating some routine and repetitive tasks, GPTs can help workers become more productive. This could boost economic growth and firm productivity. Studies estimate that AI technologies like GPTs could add an additional $15.7 trillion to the global economy by 2030.

•New types of jobs. GPTs may facilitate the creation of new jobs that utilize human judgment, creativity, and emotional intelligence. For example, AI writing assistants could help writers be more productive, but human writers and editors would still be needed. GPTs could also create demand for new data scientist and ML engineering jobs.

•Personalized services. GPTs can enable more customized and personalized services by generating tailored content for each individual user. For example, personalized education platforms and customized healthcare treatment plans. 

•Improved decision making. GPTs

In [14]:
# Save
with open('qa_system.pkl', 'wb') as f:
    pickle.dump(qa_system, f)

In [ ]:

# Load
with open('qa_system.pkl', 'rb') as f:
    qa_system = pickle.load(f)
    qa_system.client = client  # Manually set the client after loading
